# Text Generation Using distilGPT2

In [4]:
# Import packages
import tensorflow as tf
import transformers
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import os

In [ ]:
# download the pretrained DistilGPT2 model and set it to evaluation
model = GPT2LMHeadModel.from_pretrained("distilgpt2")
# model.eval()

In [ ]:
model.eval()

In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')
model = GPT2LMHeadModel.from_pretrained('distilgpt2', return_dict=True)

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
outputs = model(**inputs, labels=inputs["input_ids"])
loss = outputs.loss
logits = outputs.logits

In [ ]:
logits

In [3]:
FolderLoc = os.getcwd()

'/Users/bdavid/Documents/InsightDS/TextSimplify'